# PRUEBAS 
Responsable: Juan Alejandro Gómez Jaramillo

Repositorio: https://github.com/agomezj4/proyecto_segmentacion

In [88]:
import pandas as pd
import numpy as np
import polars as pl
import sys
import os
import yaml

In [62]:
pd.set_option('display.max_colwidth', None)

In [3]:
os.chdir('/Users/agomezj/Desktop/Data Science/Proyectos/inteligencia_digital/natural_language_processing/src')
print(os.getcwd())

/Users/agomezj/Desktop/Data Science/Proyectos/inteligencia_digital/natural_language_processing/src


In [4]:
from natural_language_processing.utils import Utils
# from proyecto_segmentacion.pipelines.raw import PipelineRaw

In [5]:
Utils.add_src_to_path()
project_root = Utils.get_project_root()

In [6]:
parameters_directory = os.path.join(project_root, 'src', 'parameters')
parameters = Utils.load_parameters(parameters_directory)

### Pipelie Raw

In [8]:
data_train_pl = Utils.load_json_pl(parameters['parameters_catalog']['train_data_path'])

In [134]:
data_train_pd = Utils.load_json_pd(parameters['parameters_catalog']['train_data_path'])

In [140]:
data_train_pl.head(1)

messages,sender_labels,receiver_labels,speakers,receivers,absolute_message_index,relative_message_index,seasons,years,game_score,game_score_delta,players,game_id
list[str],list[bool],list[str],list[str],list[str],list[i64],list[i64],list[str],list[str],list[str],list[str],list[str],i64
"[""Germany! Just the person I want to speak with. I have a somewhat crazy idea that I’ve always wanted to try with I/G, but I’ve never actually convinced the other guy to try it. And, what’s worse, it might make you suspicious of me. So...do I suggest it? I’m thinking that this is a low stakes game, not a tournament or anything, and an interesting and unusual move set might make it more fun? That’s my hope anyway. What is your appetite like for unusual and crazy?"", ""You've whet my appetite, Italy. What's the suggestion?"", … ""I really enjoyed playing with you, too! And yes, it really was my first game. Thanks for all your help and advice""]","[true, true, … true]","[""true"", ""true"", … ""true""]","[""italy"", ""germany"", … ""germany""]","[""germany"", ""italy"", … ""italy""]","[74, 76, … 2614]","[0, 1, … 320]","[""Spring"", ""Spring"", … ""Spring""]","[""1901"", ""1901"", … ""1910""]","[""3"", ""3"", … ""7""]","[""0"", ""0"", … ""-7""]","[""italy"", ""germany""]",1


In [148]:
df = data_train_pl

# Lista para almacenar los nuevos registros
new_records = []

# Iterar sobre cada fila del DataFrame
for i, row in enumerate(df.rows()):
    # Asumimos que la columna 'messages' tiene la lista de referencia
    num_elements = len(row[0])  
    for j in range(num_elements):
        new_record = {}
        # Agregar cada columna con su valor correspondiente
        for col_name in df.columns:
            col_index = df.columns.index(col_name)
            if isinstance(row[col_index], list):
                # Agregar el valor de la lista si el índice es válido
                if j < len(row[col_index]):
                    new_record[col_name] = row[col_index][j]
                else:
                    # Manejar el caso en el que la lista es más corta
                    new_record[col_name] = None
            else:
                # Replicar el valor para columnas que no son listas
                new_record[col_name] = row[col_index]
        # Agregar columna 'conversation_id' para rastrear el registro original
        new_record["conversation_id"] = i
        new_records.append(new_record)

# Crear nuevo DataFrame a partir de los registros generados
new_df = pl.DataFrame(new_records)

In [160]:
new_df

messages,sender_labels,receiver_labels,speakers,receivers,absolute_message_index,relative_message_index,seasons,years,game_score,game_score_delta,players,game_id,conversation_id
str,bool,str,str,str,i64,i64,str,str,str,str,str,i64,i64
"""Germany! Just the person I wa…",true,"""true""","""italy""","""germany""",74,0,"""Spring""","""1901""","""3""","""0""","""italy""",1,0
"""You've whet my appetite, Italy…",true,"""true""","""germany""","""italy""",76,1,"""Spring""","""1901""","""3""","""0""","""germany""",1,0
"""👍""",true,"""true""","""italy""","""germany""",86,2,"""Spring""","""1901""","""3""","""0""",null,1,0
"""It seems like there are a lot …",true,"""true""","""germany""","""italy""",87,3,"""Spring""","""1901""","""3""","""0""",null,1,0
"""Yeah, I can’t say I’ve tried i…",true,"""NOANNOTATION""","""italy""","""germany""",89,4,"""Spring""","""1901""","""3""","""0""",null,1,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Is there any way of me actuall…",true,"""true""","""france""","""england""",380,11,"""Fall""","""1902""","""4""","""-1""",null,10,187
"""Can we agree on peace? What ar…",true,"""true""","""france""","""england""",433,12,"""Fall""","""1902""","""4""","""-1""",null,10,187
"""Neutrality in exchange for cur…",true,"""false""","""england""","""france""",434,13,"""Fall""","""1902""","""5""","""1""",null,10,187
